In [17]:
from tensorflow.keras.layers import Input, SimpleRNN, LSTM, GRU, Conv1D, Embedding, Dense, Bidirectional, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial.distance import pdist, squareform
from tensorflow.keras import Model
import matplotlib.pyplot as plt
import tensorflow as tf
import random as rand
import numpy as np
import pronouncing
import markovify
import textstat
import math

In [18]:
lyric_path = '/Users/patricknaylor/Desktop/Metal/Data/lyrics_1.txt'
with open(lyric_path, 'r') as file:
    song = (file.read())
    lyrics = song.replace('\ufeff', '').split("\n")



In [19]:
markov_model = markovify.NewlineText(str("\n".join(lyrics)), well_formed=False, state_size=3)
sentence = markov_model.make_sentence(tries=100)
print(sentence)

Threw it open to the chants imminent


In [20]:
sequences = lyrics
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=20000)
tokenizer.fit_on_texts(sequences)

V = len(tokenizer.word_index)+1
seq = pad_sequences(tokenizer.texts_to_sequences(sequences), maxlen=30)
tokenizer.word_index.items()

dict_items([('the', 1), ('to', 2), ('of', 3), ('i', 4), ('and', 5), ('you', 6), ('a', 7), ('in', 8), ('your', 9), ('my', 10), ('is', 11), ('me', 12), ('for', 13), ('this', 14), ('all', 15), ('we', 16), ('will', 17), ('it', 18), ('from', 19), ('on', 20), ('with', 21), ('that', 22), ('no', 23), ('be', 24), ('now', 25), ('our', 26), ('are', 27), ('as', 28), ('by', 29), ('so', 30), ('what', 31), ('they', 32), ('life', 33), ('but', 34), ('time', 35), ('not', 36), ('see', 37), ('one', 38), ('have', 39), ('through', 40), ('out', 41), ('their', 42), ('never', 43), ('down', 44), ('can', 45), ('death', 46), ('into', 47), ('just', 48), ('when', 49), ("it's", 50), ('take', 51), ('his', 52), ("i'm", 53), ('eyes', 54), ('know', 55), ('at', 56), ('blood', 57), ('away', 58), ('up', 59), ('am', 60), ('like', 61), ('come', 62), ("don't", 63), ('us', 64), ('her', 65), ('die', 66), ('world', 67), ('was', 68), ('end', 69), ('light', 70), ('way', 71), ('has', 72), ('soul', 73), ('more', 74), ('mind', 75), (

In [21]:
train_X, train_y = seq[:, :-1], tf.keras.utils.to_categorical(seq[:, -1], num_classes=V)

print(train_X.shape, train_y.shape)

(57446, 29) (57446, 20761)


In [22]:
D = 512

#Simple RNN
T = train_X.shape[1]
i = Input(shape=(T,))
x = Embedding(V, D)(i)
x = Dropout(0.2)(x)
x = SimpleRNN(150)(x)
x = Dense(V, activation="softmax")(x)
rnn_model = Model(i, x)

adam = tf.keras.optimizers.Adam(learning_rate=0.001)

rnn_model.compile(optimizer=adam, metrics=["accuracy"], loss="categorical_crossentropy")
rnn_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 29)]              0         
                                                                 
 embedding_1 (Embedding)     (None, 29, 512)           10629632  
                                                                 
 dropout_1 (Dropout)         (None, 29, 512)           0         
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 150)               99450     
                                                                 
 dense_1 (Dense)             (None, 20761)             3134911   
                                                                 
Total params: 13863993 (52.89 MB)
Trainable params: 13863993 (52.89 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
from keras.callbacks import ReduceLROnPlateau , EarlyStopping
from tensorflow.keras.optimizers import Adam 


import warnings
warnings.filterwarnings('ignore')
# Set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

es = EarlyStopping(monitor="loss", mode="min", verbose=1, patience=20)

In [24]:
rnn_r = rnn_model.fit(train_X, train_y, epochs=100,callbacks=[learning_rate_reduction,es])

Epoch 1/100
1795/1796 [============================>.] - ETA: 0s - loss: 8.0471 - accuracy: 0.0261WARNING:tensorflow:Learning rate reduction is conditioned on metric `val_accuracy` which is not available. Available metrics are: loss,accuracy,lr


1796/1796 [==============================] - 69s 38ms/step - loss: 8.0471 - accuracy: 0.0262 - lr: 0.0010
Epoch 2/100
1796/1796 [==============================] - ETA: 0s - loss: 6.5982 - accuracy: 0.0766WARNING:tensorflow:Learning rate reduction is conditioned on metric `val_accuracy` which is not available. Available metrics are: loss,accuracy,lr


1796/1796 [==============================] - 68s 38ms/step - loss: 6.5982 - accuracy: 0.0766 - lr: 0.0010
Epoch 3/100
1795/1796 [============================>.] - ETA: 0s - loss: 5.5204 - accuracy: 0.1553WARNING:tensorflow:Learning rate reduction is conditioned on metric `val_accuracy` which is not available. Available metrics are: loss,accuracy,lr


1796/1796 [==============================] - 66s 36ms/step - loss: 5.5204 - accuracy: 0.1554 - lr: 0.0010
Epoch 4/100
1795/1796 [============================>.] - ETA: 0s - loss: 4.5529 - accuracy: 0.2519WARNING:tensorflow:Learning rate reduction is conditioned on metric `val_accuracy` which is not available. Available metrics are: loss,accuracy,lr


1796/1796 [==============================] - 65s 36ms/step - loss: 4.5528 - accuracy: 0.2519 - lr: 0.0010
Epoch 5/100
1796/1796 [==============================] - ETA: 0s - loss: 3.7350 - accuracy: 0.3586WARNING:tensorflow:Learning rate reduction is conditioned on metric `val_accuracy` which is not available. Available metrics are: loss,accuracy,lr


...

1796/1796 [==============================] - 69s 38ms/step - loss: 0.8432 - accuracy: 0.8167 - lr: 0.0010
Epoch 50/100
1796/1796 [==============================] - ETA: 0s - loss: 0.8296 - accuracy: 0.8210WARNING:tensorflow:Learning rate reduction is conditioned on metric `val_accuracy` which is not available. Available metrics are: loss,accuracy,lr


1796/1796 [==============================] - 68s 38ms/step - loss: 0.8296 - accuracy: 0.8210 - lr: 0.0010
Epoch 51/100
1796/1796 [==============================] - ETA: 0s - loss: 0.8420 - accuracy: 0.8176WARNING:tensorflow:Learning rate reduction is conditioned on metric `val_accuracy` which is not available. Available metrics are: loss,accuracy,lr


1796/1796 [==============================] - 68s 38ms/step - loss: 0.8420 - accuracy: 0.8176 - lr: 0.0010
Epoch 52/100
1795/1796 [============================>.] - ETA: 0s - loss: 0.8377 - accuracy: 0.8172WARNING:tensorflow:Learning rate reduction is conditioned on metric `val_accuracy` which is not available. Available metrics are: loss,accuracy,lr


1796/1796 [==============================] - 69s 38ms/step - loss: 0.8376 - accuracy: 0.8172 - lr: 0.0010
Epoch 53/100
1796/1796 [==============================] - ETA: 0s - loss: 0.8314 - accuracy: 0.8183WARNING:tensorflow:Learning rate reduction is conditioned on metric `val_accuracy` which is not available. Available metrics are: loss,accuracy,lr


1796/1796 [==============================] - 69s 38ms/step - loss: 0.8314 - accuracy: 0.8183 - lr: 0.0010
Epoch 54/100
 195/1796 [==>...........................] - ETA: 1:04 - loss: 0.7166 - accuracy: 0.8420

In [ ]:
def calc_readability(input_bars):
  avg_readability = 0
  for bar in input_bars:
    avg_readability += textstat.automated_readability_index(bar)
  return avg_readability / len(input_bars)

In [ ]:
def calc_rhyme_density(bars):
  total_syllables = 0
  rhymed_syllables = 0
  for bar in bars:
    for word in bar.split():
      p = pronouncing.phones_for_word(word)
      if len(p) == 0:
        break
      syllables = pronouncing.syllable_count(p[0])
      total_syllables += syllables
      has_rhyme = False
      for rhyme in pronouncing.rhymes(word):
        if has_rhyme:
          break
        for idx, r_bar in enumerate(bars):
          if idx > 4:
            break
          if rhyme in r_bar:
            rhymed_syllables += syllables
            has_rhyme = True
            break
  return rhymed_syllables/total_syllables

In [ ]:
def generate_bar(seed_phrase, model, length_of_bar):
  for i in range(length_of_bar):
    seed_tokens = pad_sequences(tokenizer.texts_to_sequences([seed_phrase]), maxlen=29)
    output_p = model.predict(seed_tokens)
    output_word = np.argmax(output_p, axis=1)[0]-1
    seed_phrase += " " + str(list(tokenizer.word_index.items())[output_word][0])
  return seed_phrase

In [ ]:
def score_bar(input_bar, artists_bars, artists_avg_readability, artists_avg_rhyme_idx):
  gen_readability = textstat.automated_readability_index(input_bar)
  gen_rhyme_idx = calc_rhyme_density(input_bar)
  comp_bars = compare_bars(input_bar, artists_bars)

  # Scores based off readability, rhyme index, and originality. The lower the score the better.
  bar_score = (artists_avg_readability - gen_readability) + (artists_avg_rhyme_idx - gen_rhyme_idx) + comp_bars
  return bar_score

In [ ]:
def compare_bars(input_bar, artists_bars):
  '''
    input_bars are the fire bars our AI generates
    artists_bars are the original bars for the artist

    The lower the score the better! We want unique bars
  '''
  # Converts sentences to matrix of token counts
  avg_dist = 0
  total_counted = 0
  for bar in artists_bars:
    v = CountVectorizer()
    # Vectorize the sentences
    word_vector = v.fit_transform([input_bar, bar])
    # Compute the cosine distance between the sentence vectors
    cos_dist = 1-pdist(word_vector.toarray(), 'cosine')[0]
    if not math.isnan(cos_dist):
      avg_dist += 1-pdist(word_vector.toarray(), 'cosine')[0]
      total_counted += 1
  return avg_dist/total_counted

In [ ]:
def generate_song( model, artists_bars, length_of_bar=10, length_of_song=20, min_score_threshold=-0.2, max_score_threshold=0.2, tries=5):
  artists_avg_readability = calc_readability(artists_bars)
  artists_avg_rhyme_idx = calc_rhyme_density(artists_bars)
  fire_song = []
  cur_tries = 0
  candidate_bars = []

  while len(fire_song) < length_of_song:
    try:
        seed_sentence = markov_model.make_sentence(tries=100).split(" ")
        seed_sentence = " ".join(seed_sentence[:5])
    except:
        pass
    cur_tries += 1
    bar = generate_bar(seed_sentence, model, rand.randrange(5, length_of_bar))
    bar_score = score_bar(bar, artist_lyrics, artists_avg_readability, artists_avg_rhyme_idx) 
    candidate_bars.append((bar_score, bar))


    if bar_score <= max_score_threshold and bar_score >= min_score_threshold:
      fire_song.append(bar)
      cur_tries = 0
      print("Generated Bar: ", len(fire_song))

    if cur_tries >= tries:
      lowest_score = np.Infinity
      best_bar = ""
      for bar in candidate_bars:
        if bar[0] < lowest_score:
          best_bar = bar[1]
          candidate_bars = []
      
      fire_song.append(best_bar)
      print("Generated Bar: ", len(fire_song))
      cur_tries = 0
      
  print("Generated song with avg rhyme density: ", calc_rhyme_density(fire_song), "and avg readability of: ", calc_readability(fire_song))
  return fire_song

In [ ]:
rnn = generate_song( rnn_model, artist_lyrics, length_of_bar =10 , tries=100)

print("Song Generated with SimpleRNN:")
for line in rnn:
  print(line)
print()